In [31]:
import numpy as np
import pandas as pd

## a)

In [12]:
S = np.array([.160, .203, .248, .271, .210, .200, .187])

Cor = [[1,     0.488, 0.478, 0.515, 0.439, 0.512, 0.491],
       [0.488, 1,     0.664, 0.655, 0.310, 0.608, 0.779],
       [0.478, 0.664, 1,     0.861, 0.355, 0.783, 0.668],
       [0.515, 0.655, 0.861, 1,     0.354, 0.777, 0.653],
       [0.439, 0.310, 0.355, 0.354, 1,     0.405, 0.306],
       [0.512, 0.608, 0.783, 0.777, 0.405, 1,     0.652],
       [0.491, 0.779, 0.668, 0.653, 0.306, 0.652, 1]]

Sigma = np.diag(S) @ Cor @ np.diag(S)

w_eq = np.transpose(np.array([.016, .022, .052, .055, .116, .124, .615]))

In [13]:
# a)
mu_0 = 3.5 * np.matmul(Sigma, w_eq)
mu_0

array([0.05512576, 0.09681265, 0.11701321, 0.12638136, 0.06023934,
       0.0947477 , 0.10584065])

## b)

In [15]:
tau = 0.03
w2 = np.linalg.inv(2*(1+tau) * Sigma) @ mu_0
print("weights: " + "\n" + str(w2) + "\n")
print("difference:" + "\n" + str(abs(w2-w_eq)))

weights: 
[0.02718447 0.03737864 0.08834951 0.0934466  0.19708738 0.21067961
 1.04490291]

difference:
[0.01118447 0.01537864 0.03634951 0.0384466  0.08108738 0.08667961
 0.42990291]


## c)

In [72]:
P@((1+tau)*Sigma)

array([[ 0.00533873,  0.00902834,  0.01270213,  0.0274736 ,  0.00278032,
         0.00253078,  0.00696171],
       [ 0.00119431,  0.0119865 ,  0.00252273,  0.00302968,  0.00123464,
         0.000309  , -0.0055593 ]])

In [19]:
w_fr = w_eq[2]/(w_eq[5]+w_eq[2])
w_uk = w_eq[5]/(w_eq[5]+w_eq[2])
Omega = np.array([[0.025*tau, 0],
                  [, 0.015*tau]])
q = np.transpose(np.array([0.045, 0.02]))
P = np.array([[0, 0, -w_fr, 1, 0, -w_uk, 0],
              [0, 1, 0, 0, 0, 0, -1]])

In [20]:
mu_hat = np.linalg.inv(np.linalg.inv(tau*Sigma) + np.transpose(P) @ np.linalg.inv(Omega) @ P) \
@ (np.linalg.inv(tau*Sigma) @ mu_0 + np.transpose(P) @ np.linalg.inv(Omega) @ q)

Sigma_hat = Sigma + np.linalg.inv(np.linalg.inv(tau*Sigma) + np.transpose(P) @ np.linalg.inv(Omega) @ P)

## d)

In [26]:
w_opt = np.linalg.inv(2*Sigma_hat) @ mu_hat

In [32]:
countries = ['Australia','Canada','France','Germany','Japan','UK','USA']

In [61]:
weights = pd.DataFrame({'w_market':w_eq, 'w_prior':w2, 'w_posterior2':w_opt}, index = countries)

In [62]:
weights

,w_market,w_prior,w_posterior2
Australia,0.016,0.027184,0.027184
Canada,0.022,0.037379,0.467688
France,0.052,0.088350,0.028856
Germany,0.055,0.093447,0.294810
Japan,0.116,0.197087,0.197087
UK,0.124,0.210680,0.068810
USA,0.615,1.044903,0.614593


We see that by incorporating the view that Germany will outperform a market weighted France and UK, the posterior weight of Germany dramatically increases comparing to its prior weight, whereas that of France and UK decrease significantly in the posterior. Moreover, the view that Canada will outperform the US make their posterior weight dramatically increase and decrease respectively. 

## e)

In [97]:
Lambda = 1/2 * np.linalg.inv( P @ Sigma@ P.T + (1+1/tau)*Omega ) @ ( q - (1/(1+tau))*(P @ mu_0) )

In [100]:
print('The view weights are:\nlambda_1 = {:.4}\nlambda_2 = {:.4}'.format(Lambda[0],Lambda[1]))

The view weights are:
lambda_1 = 0.2014
lambda_2 = 0.4303


## f)

In [87]:
P_jap = np.array([[0,0,0,0,1,0,0]])
P3 = np.concatenate((P, P_jap), 0)
q3 = np.append(q, 0.055)
Omega3 = np.array([[0.025*tau, 0, 0],
                   [0, 0.015*tau, 0],
                   [0, 0, 0.04*tau]])

In [88]:
mu_hat3 = np.linalg.inv(np.linalg.inv(tau*Sigma) + np.transpose(P3) @ np.linalg.inv(Omega3) @ P3) \
@ (np.linalg.inv(tau*Sigma) @ mu_0 + np.transpose(P3) @ np.linalg.inv(Omega3) @ q3)

Sigma_hat3 = Sigma + np.linalg.inv(np.linalg.inv(tau*Sigma) + np.transpose(P3) @ np.linalg.inv(Omega3) @ P3)

In [89]:
w_opt3 = np.linalg.inv(2*Sigma_hat3) @ mu_hat3

In [90]:
weights['w_posterior3'] = w_opt3
weights

,w_market,w_prior,w_posterior2,w_posterior3
Australia,0.016,0.027184,0.027184,0.027184
Canada,0.022,0.037379,0.467688,0.468789
France,0.052,0.088350,0.028856,0.028312
Germany,0.055,0.093447,0.294810,0.296652
Japan,0.116,0.197087,0.197087,0.164168
UK,0.124,0.210680,0.068810,0.067512
USA,0.615,1.044903,0.614593,0.613492


Since the view on Japan of 5.5% is lower than the prior on Japan of ~6%, its posterior weight decreases from 0.19 to 0.16. If we reduce Japan's view uncertainty from 0.04 to 0.01, we expect this decrease to be even more pronouced.

In [92]:
Omega3_2 = np.array([[0.025*tau, 0, 0],
                   [0, 0.015*tau, 0],
                   [0, 0, 0.01*tau]])
mu_hat3_2 = np.linalg.inv(np.linalg.inv(tau*Sigma) + np.transpose(P3) @ np.linalg.inv(Omega3_2) @ P3) \
@ (np.linalg.inv(tau*Sigma) @ mu_0 + np.transpose(P3) @ np.linalg.inv(Omega3_2) @ q3)
Sigma_hat3_2 = Sigma + np.linalg.inv(np.linalg.inv(tau*Sigma) + np.transpose(P3) @ np.linalg.inv(Omega3_2) @ P3)
w_opt3_2 = np.linalg.inv(2*Sigma_hat3_2) @ mu_hat3_2
weights['w_posterior3_2'] = w_opt3_2
weights

,w_market,w_prior,w_posterior2,w_posterior3,w_posterior3_2
Australia,0.016,0.027184,0.027184,0.027184,0.027184
Canada,0.022,0.037379,0.467688,0.468789,0.469417
France,0.052,0.088350,0.028856,0.028312,0.028001
Germany,0.055,0.093447,0.294810,0.296652,0.297702
Japan,0.116,0.197087,0.197087,0.164168,0.145403
UK,0.124,0.210680,0.068810,0.067512,0.066773
USA,0.615,1.044903,0.614593,0.613492,0.612865


Indeed, we see that reducing the uncertainty on Japan's view decreases its weight even more from 0.16 to 0.14.

In [102]:
Lambda3 = 1/2 * np.linalg.inv( P3 @ Sigma @ P3.T + (1+1/tau)*Omega3 ) @ ( q3 - (1/(1+tau))*(P3 @ mu_0) )

In [104]:
print('The view weights are:\nlambda_1 = {:.4}\nlambda_2 = {:.4}\nlambda_3 = {:.4}'.format(Lambda3[0],Lambda3[1],Lambda3[2]))

The view weights are:
lambda_1 = 0.2032
lambda_2 = 0.4314
lambda_3 = -0.03292
